In [75]:
import json
import os
import numpy as np
from astropy.table import Table
from astropy.io import fits
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.wcs import WCS
from astropy import wcs
from astropy.table import Table
import time
import requests
import shutil # save img locally
import h5py

In [76]:
#print table to view data
SGA = Table.read('SGA_MorphsOnly.fits')
SGA

SGA_ID,SGA_GALAXY,GALAXY,PGC,RA_LEDA,DEC_LEDA,MORPHTYPE,PA_LEDA,D25_LEDA,BA_LEDA,Z_LEDA,SB_D25_LEDA,MAG_LEDA,BYHAND,REF,GROUP_ID,GROUP_NAME,GROUP_MULT,GROUP_PRIMARY,GROUP_RA,GROUP_DEC,GROUP_DIAMETER,BRICKNAME,RA,DEC,D26,D26_REF,PA,BA,RA_MOMENT,DEC_MOMENT,SMA_MOMENT,G_SMA50,R_SMA50,Z_SMA50,SMA_SB22,SMA_SB22.5,SMA_SB23,SMA_SB23.5,SMA_SB24,SMA_SB24.5,SMA_SB25,SMA_SB25.5,SMA_SB26,G_MAG_SB22,R_MAG_SB22,Z_MAG_SB22,G_MAG_SB22.5,R_MAG_SB22.5,Z_MAG_SB22.5,G_MAG_SB23,R_MAG_SB23,Z_MAG_SB23,G_MAG_SB23.5,R_MAG_SB23.5,Z_MAG_SB23.5,G_MAG_SB24,R_MAG_SB24,Z_MAG_SB24,G_MAG_SB24.5,R_MAG_SB24.5,Z_MAG_SB24.5,G_MAG_SB25,R_MAG_SB25,Z_MAG_SB25,G_MAG_SB25.5,R_MAG_SB25.5,Z_MAG_SB25.5,G_MAG_SB26,R_MAG_SB26,Z_MAG_SB26,SMA_SB22_ERR,SMA_SB22.5_ERR,SMA_SB23_ERR,SMA_SB23.5_ERR,SMA_SB24_ERR,SMA_SB24.5_ERR,SMA_SB25_ERR,SMA_SB25.5_ERR,SMA_SB26_ERR,G_MAG_SB22_ERR,R_MAG_SB22_ERR,Z_MAG_SB22_ERR,G_MAG_SB22.5_ERR,R_MAG_SB22.5_ERR,Z_MAG_SB22.5_ERR,G_MAG_SB23_ERR,R_MAG_SB23_ERR,Z_MAG_SB23_ERR,G_MAG_SB23.5_ERR,R_MAG_SB23.5_ERR,Z_MAG_SB23.5_ERR,G_MAG_SB24_ERR,R_MAG_SB24_ERR,Z_MAG_SB24_ERR,G_MAG_SB24.5_ERR,R_MAG_SB24.5_ERR,Z_MAG_SB24.5_ERR,G_MAG_SB25_ERR,R_MAG_SB25_ERR,Z_MAG_SB25_ERR,G_MAG_SB25.5_ERR,R_MAG_SB25.5_ERR,Z_MAG_SB25.5_ERR,G_MAG_SB26_ERR,R_MAG_SB26_ERR,Z_MAG_SB26_ERR,G_COG_PARAMS_MTOT,G_COG_PARAMS_M0,G_COG_PARAMS_ALPHA1,G_COG_PARAMS_ALPHA2,G_COG_PARAMS_CHI2,R_COG_PARAMS_MTOT,R_COG_PARAMS_M0,R_COG_PARAMS_ALPHA1,R_COG_PARAMS_ALPHA2,R_COG_PARAMS_CHI2,Z_COG_PARAMS_MTOT,Z_COG_PARAMS_M0,Z_COG_PARAMS_ALPHA1,Z_COG_PARAMS_ALPHA2,Z_COG_PARAMS_CHI2,ELLIPSEBIT,FLUX_G,FLUX_R,FLUX_Z,PSFDEPTH_G,PSFDEPTH_R,PSFDEPTH_Z,PSFSIZE_G,PSFSIZE_R,PSFSIZE_Z,RELEASE,BRICKID,OBJID,MASKBITS,SOURCE_TYPE,EBV,FIBERFLUX_G,FIBERFLUX_R,FIBERFLUX_Z,NOBS_G,NOBS_R,NOBS_Z,FRACIN_G,FRACIN_R,FRACIN_Z,HUBBLE_TYPE
int64,bytes16,bytes29,int64,float64,float64,bytes21,float32,float32,float32,float32,float32,float32,bool,bytes13,int64,bytes35,int16,bool,float64,float64,float32,bytes8,float64,float64,float32,bytes4,float32,float32,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16,int32,int32,int16,bytes6,float32,float32,float32,float32,int16,int16,int16,float32,float32,float32,float64
284583,SGA-2020 284583,PGC1753576,1753576,246.63893700000006,25.882395000000002,E,83.58,0.4168694,0.61659503,0.049369153,23.009481,16.281,False,LEDA-20181114,102768,2MASXJ16263037+2552589_GROUP,3,False,246.63132430980986,25.886678155342814,0.92581606,2465p260,246.63899443421053,25.88239593752038,1.0317777,SB26,89.25743,0.7660097,246.63894134159338,25.882421271706928,21.809486,7.683644,7.267518,6.908575,6.584644,8.19172,10.043491,12.2072115,14.66538,17.695236,21.416355,25.87598,30.953333,16.50979,15.567412,14.869183,16.380758,15.444356,14.747147,16.275633,15.342991,14.648945,16.190153,15.260059,14.5695095,16.121006,15.193452,14.5059595,16.06036,15.134916,14.4499445,16.00693,15.085361,14.402035,15.962352,15.040784,14.35771,15.920147,15.0014,14.320146,0.030591438,0.04308892,0.045999907,0.04829483,0.06055945,0.080751404,0.0973,0.124484144,0.20583051,0.01532438,0.015401917,0.015267819,0.013852132,0.013975341,0.01385223,0.012736728,0.012889164,0.012807729,0.0118285,0.011991284,0.011952993,0.011088653,0.011258294,0.011255878,0.01050395,0.010681791,0.010707616,0.010012423,

In [77]:
#Command to retrieve url datas
# RETURNS JPEG IMAGES, FOR VIEWING ONLY
def get_jpgcut(galid,ra,dec):
    url = 'https://www.legacysurvey.org/viewer/cutout.jpg?ra={}&dec={}&%22/pix=0.25&layer=ls-dr9&size=152'.format(ra, dec)
    print(url)
    res = requests.get(url, stream = True)
    file_name = 'Images/Gal{}.jpg'.format(galid)
    
    if res.status_code == 200:
        with open(file_name,'wb') as f:
            shutil.copyfileobj(res.raw, f)
        print('Image sucessfully Downloaded: ',file_name)
    else:
        print('Image Couldn\'t be retrieved')

In [110]:
#Command to retrieve url datas for FITS files
def get_fitscut(galid,ra,dec):
    url = 'https://www.legacysurvey.org/viewer/cutout.fits?ra={}&dec={}&%22/pix=0.25&layer=ls-dr9&size=152'.format(ra, dec)
    #print(url)
    res = requests.get(url, stream = True)
    file_name = 'FITS/Gal{}.fits'.format(galid)
    
    if res.status_code == 200:
        with open(file_name,'wb') as f:
            shutil.copyfileobj(res.raw, f)
        #print('Image sucessfully Downloaded: ',file_name)
    else:
        print('Image Couldn\'t be retrieved')

In [51]:
#create some pretty pictures
i=0
while i < 10:
    get_jpgcut(SGA['SGA_ID'][i],SGA['RA_LEDA'][i],SGA['DEC_LEDA'][i])
    time.sleep(1)
    i+=1

https://www.legacysurvey.org/viewer/cutout.jpg?ra=206.85561899999996&dec=59.754215&%22/pix=0.25&layer=ls-dr9&size=152
Image sucessfully Downloaded:  Images/Gal548100.jpg
https://www.legacysurvey.org/viewer/cutout.jpg?ra=175.20097499999997&dec=33.5209457&%22/pix=0.25&layer=ls-dr9&size=152
Image sucessfully Downloaded:  Images/Gal450699.jpg
https://www.legacysurvey.org/viewer/cutout.jpg?ra=178.11407100000002&dec=34.8944672&%22/pix=0.25&layer=ls-dr9&size=152
Image sucessfully Downloaded:  Images/Gal657041.jpg
https://www.legacysurvey.org/viewer/cutout.jpg?ra=247.95205049999998&dec=19.4742468&%22/pix=0.25&layer=ls-dr9&size=152
Image sucessfully Downloaded:  Images/Gal1325951.jpg
https://www.legacysurvey.org/viewer/cutout.jpg?ra=196.231608&dec=16.582039&%22/pix=0.25&layer=ls-dr9&size=152
Image sucessfully Downloaded:  Images/Gal355862.jpg
https://www.legacysurvey.org/viewer/cutout.jpg?ra=116.1348405&dec=33.0954033&%22/pix=0.25&layer=ls-dr9&size=152
Image sucessfully Downloaded:  Images/Gal8

In [107]:
f = h5py.File('cutoutdata.hdf5','a')

In [108]:
count = 5500
brickid = f.create_dataset('brickid',(count,), maxshape=(None,))
flux_g = f.create_dataset('flux_g',(count,), maxshape=(None,))
flux_r = f.create_dataset('flux_r',(count,), maxshape=(None,))
flux_z = f.create_dataset('flux_z',(count,), maxshape=(None,))
fiberflux_g = f.create_dataset('fiberflux_g',(count,), maxshape=(None,))
fiberflux_r = f.create_dataset('fiberflux_r',(count,), maxshape=(None,))
fiberflux_z = f.create_dataset('fiberflux_z',(count,), maxshape=(None,))
psfsize_g = f.create_dataset('psfsize_g',(count,), maxshape=(None,))
psfsize_r = f.create_dataset('psfsize_r',(count,), maxshape=(None,))
psfsize_z = f.create_dataset('psfsize_z',(count,), maxshape=(None,))
psfdepth_g = f.create_dataset('psfdepth_g',(count,), maxshape=(None,))
psfdepth_r = f.create_dataset('psfdepth_r',(count,), maxshape=(None,))
psfdepth_z = f.create_dataset('psfdepth_z',(count,), maxshape=(None,))
nobs_g = f.create_dataset('nobs_g',(count,), maxshape=(None,))
nobs_r = f.create_dataset('nobs_r',(count,), maxshape=(None,))
nobs_z = f.create_dataset('nobs_z',(count,), maxshape=(None,))
fracin_g = f.create_dataset('fracin_g',(count,), maxshape=(None,))
fracin_r = f.create_dataset('fracin_r',(count,), maxshape=(None,))
fracin_z = f.create_dataset('fracin_z',(count,), maxshape=(None,))
release = f.create_dataset('release',(count,), maxshape=(None,))
objid = f.create_dataset('objid',(count,), maxshape=(None,))
source_type = f.create_dataset('source_type',(count,), maxshape=(None,))
maskbits = f.create_dataset('maskbits',(count,), maxshape=(None,))
ebv = f.create_dataset('ebv',(count,), maxshape=(None,))
images = f.create_dataset('images',(count,3,152,152,), maxshape=(None, 3, 152, 152))
ra = f.create_dataset('ra',(count,), maxshape=(None,))
dec = f.create_dataset('dec',(count,), maxshape=(None,))
mag_g = f.create_dataset('mag_g',(count,), maxshape=(None,))
mag_r = f.create_dataset('mag_r',(count,), maxshape=(None,))
mag_z = f.create_dataset('mag_z',(count,), maxshape=(None,))
#PROBLEMATIC ONE:
z_phot_median = f.create_dataset('z_phot_median',(count,), maxshape=(None,))

In [116]:
count_per_type = int((count/11))
print(count_per_type)

500


In [111]:
#collect data for (MORPHTYPE= -5)
i=0 #controls location within SGA catalog
j=0 #controls location within the h5 file, separate so you cacn skip around in catalog to make sure all morphologies are represented
while i < 500:
    get_fitscut(SGA['SGA_ID'][i],SGA['RA_LEDA'][i],SGA['DEC_LEDA'][i])
    img = fits.open('FITS/Gal{}.fits'.format(SGA['SGA_ID'][i]))
    images[j] = np.array(img[0].data)
    
    # Compile additional values into h5 file
    brickid[j] = SGA['BRICKID'][i]
    flux_g[j] = SGA['FLUX_G'][i]
    flux_r[j] = SGA['FLUX_R'][i]
    flux_z[j] = SGA['FLUX_Z'][i]
    fiberflux_g[j] = SGA['FIBERFLUX_G'][i]
    fiberflux_r[j] = SGA['FIBERFLUX_R'][i]
    fiberflux_z[j] = SGA['FIBERFLUX_R'][i]
    psfsize_g[j] = SGA['PSFSIZE_G'][i]
    psfsize_r[j] = SGA['PSFSIZE_R'][i]
    psfsize_z[j] = SGA['PSFSIZE_Z'][i] 
    psfdepth_g[j] = SGA['PSFDEPTH_G'][i] 
    psfdepth_r[j] = SGA['PSFDEPTH_R'][i]
    psfdepth_z[j] = SGA['PSFDEPTH_Z'][i] 
    nobs_g[j] = SGA['NOBS_G'][i]
    nobs_r[j] = SGA['NOBS_R'][i] 
    nobs_z[j] = SGA['NOBS_Z'][i] 
    fracin_g[j] = SGA['FRACIN_G'][i] 
    fracin_r[j] = SGA['FRACIN_R'][i] 
    fracin_z[j] = SGA['FRACIN_Z'][i]
    release[j] = SGA['RELEASE'][i]
    objid[j] = SGA['OBJID'][i]
    ebv[j] = SGA['EBV'][i]
    ra[j] = SGA['RA'][i]
    dec[j] = SGA['DEC'][i]
    mag_g[j] = SGA['G_MAG_SB26'][i]
    mag_r[j] = SGA['R_MAG_SB26'][i]
    mag_z[j] = SGA['Z_MAG_SB26'][i]
    
    time.sleep(1) #1 second pause per loop to not overwhelm server
    
    i+=1
    j+=1

In [117]:
#collect data for (MORPHTYPE = -2)
i=45468 #controls location within SGA catalog
j=(count_per_type*1) #controls location within the h5 file, separate so you cacn skip around in catalog to make sure all morphologies are represented
while i < (45468+count_per_type):
    get_fitscut(SGA['SGA_ID'][i],SGA['RA_LEDA'][i],SGA['DEC_LEDA'][i])
    img = fits.open('FITS/Gal{}.fits'.format(SGA['SGA_ID'][i]))
    images[j] = np.array(img[0].data)
    
    # Compile additional values into h5 file
    brickid[j] = SGA['BRICKID'][i]
    flux_g[j] = SGA['FLUX_G'][i]
    flux_r[j] = SGA['FLUX_R'][i]
    flux_z[j] = SGA['FLUX_Z'][i]
    fiberflux_g[j] = SGA['FIBERFLUX_G'][i]
    fiberflux_r[j] = SGA['FIBERFLUX_R'][i]
    fiberflux_z[j] = SGA['FIBERFLUX_R'][i]
    psfsize_g[j] = SGA['PSFSIZE_G'][i]
    psfsize_r[j] = SGA['PSFSIZE_R'][i]
    psfsize_z[j] = SGA['PSFSIZE_Z'][i] 
    psfdepth_g[j] = SGA['PSFDEPTH_G'][i] 
    psfdepth_r[j] = SGA['PSFDEPTH_R'][i]
    psfdepth_z[j] = SGA['PSFDEPTH_Z'][i] 
    nobs_g[j] = SGA['NOBS_G'][i]
    nobs_r[j] = SGA['NOBS_R'][i] 
    nobs_z[j] = SGA['NOBS_Z'][i] 
    fracin_g[j] = SGA['FRACIN_G'][i] 
    fracin_r[j] = SGA['FRACIN_R'][i] 
    fracin_z[j] = SGA['FRACIN_Z'][i]
    release[j] = SGA['RELEASE'][i]
    objid[j] = SGA['OBJID'][i]
    ebv[j] = SGA['EBV'][i]
    ra[j] = SGA['RA'][i]
    dec[j] = SGA['DEC'][i]
    mag_g[j] = SGA['G_MAG_SB26'][i]
    mag_r[j] = SGA['R_MAG_SB26'][i]
    mag_z[j] = SGA['Z_MAG_SB26'][i]
    
    time.sleep(1) #1 second pause per loop to not overwhelm server
    
    i+=1
    j+=1

In [118]:
#collect data for (MORPHTYPE = 0)
i=50217 #controls location within SGA catalog
j=(count_per_type*2) #controls location within the h5 file, separate so you cacn skip around in catalog to make sure all morphologies are represented
while i < (50217+count_per_type):
    get_fitscut(SGA['SGA_ID'][i],SGA['RA_LEDA'][i],SGA['DEC_LEDA'][i])
    img = fits.open('FITS/Gal{}.fits'.format(SGA['SGA_ID'][i]))
    images[j] = np.array(img[0].data)
    
    # Compile additional values into h5 file
    brickid[j] = SGA['BRICKID'][i]
    flux_g[j] = SGA['FLUX_G'][i]
    flux_r[j] = SGA['FLUX_R'][i]
    flux_z[j] = SGA['FLUX_Z'][i]
    fiberflux_g[j] = SGA['FIBERFLUX_G'][i]
    fiberflux_r[j] = SGA['FIBERFLUX_R'][i]
    fiberflux_z[j] = SGA['FIBERFLUX_R'][i]
    psfsize_g[j] = SGA['PSFSIZE_G'][i]
    psfsize_r[j] = SGA['PSFSIZE_R'][i]
    psfsize_z[j] = SGA['PSFSIZE_Z'][i] 
    psfdepth_g[j] = SGA['PSFDEPTH_G'][i] 
    psfdepth_r[j] = SGA['PSFDEPTH_R'][i]
    psfdepth_z[j] = SGA['PSFDEPTH_Z'][i] 
    nobs_g[j] = SGA['NOBS_G'][i]
    nobs_r[j] = SGA['NOBS_R'][i] 
    nobs_z[j] = SGA['NOBS_Z'][i] 
    fracin_g[j] = SGA['FRACIN_G'][i] 
    fracin_r[j] = SGA['FRACIN_R'][i] 
    fracin_z[j] = SGA['FRACIN_Z'][i]
    release[j] = SGA['RELEASE'][i]
    objid[j] = SGA['OBJID'][i]
    ebv[j] = SGA['EBV'][i]
    ra[j] = SGA['RA'][i]
    dec[j] = SGA['DEC'][i]
    mag_g[j] = SGA['G_MAG_SB26'][i]
    mag_r[j] = SGA['R_MAG_SB26'][i]
    mag_z[j] = SGA['Z_MAG_SB26'][i]
    
    time.sleep(0.1) #1 second pause per loop to not overwhelm server
    
    i+=1
    j+=1

In [119]:
#collect data for (MORPHTYPE = 1)
i=64744 #controls location within SGA catalog
j=(count_per_type*3) #controls location within the h5 file, separate so you cacn skip around in catalog to make sure all morphologies are represented
while i < (64744+count_per_type):
    get_fitscut(SGA['SGA_ID'][i],SGA['RA_LEDA'][i],SGA['DEC_LEDA'][i])
    img = fits.open('FITS/Gal{}.fits'.format(SGA['SGA_ID'][i]))
    images[j] = np.array(img[0].data)
    
    # Compile additional values into h5 file
    brickid[j] = SGA['BRICKID'][i]
    flux_g[j] = SGA['FLUX_G'][i]
    flux_r[j] = SGA['FLUX_R'][i]
    flux_z[j] = SGA['FLUX_Z'][i]
    fiberflux_g[j] = SGA['FIBERFLUX_G'][i]
    fiberflux_r[j] = SGA['FIBERFLUX_R'][i]
    fiberflux_z[j] = SGA['FIBERFLUX_R'][i]
    psfsize_g[j] = SGA['PSFSIZE_G'][i]
    psfsize_r[j] = SGA['PSFSIZE_R'][i]
    psfsize_z[j] = SGA['PSFSIZE_Z'][i] 
    psfdepth_g[j] = SGA['PSFDEPTH_G'][i] 
    psfdepth_r[j] = SGA['PSFDEPTH_R'][i]
    psfdepth_z[j] = SGA['PSFDEPTH_Z'][i] 
    nobs_g[j] = SGA['NOBS_G'][i]
    nobs_r[j] = SGA['NOBS_R'][i] 
    nobs_z[j] = SGA['NOBS_Z'][i] 
    fracin_g[j] = SGA['FRACIN_G'][i] 
    fracin_r[j] = SGA['FRACIN_R'][i] 
    fracin_z[j] = SGA['FRACIN_Z'][i]
    release[j] = SGA['RELEASE'][i]
    objid[j] = SGA['OBJID'][i]
    ebv[j] = SGA['EBV'][i]
    ra[j] = SGA['RA'][i]
    dec[j] = SGA['DEC'][i]
    mag_g[j] = SGA['G_MAG_SB26'][i]
    mag_r[j] = SGA['R_MAG_SB26'][i]
    mag_z[j] = SGA['Z_MAG_SB26'][i]
    
    time.sleep(0.1) #1 second pause per loop to not overwhelm server
    
    i+=1
    j+=1

In [120]:
#collect data for (MORPHTYPE = 2)
i=67631 #controls location within SGA catalog
j=(count_per_type*4) #controls location within the h5 file, separate so you cacn skip around in catalog to make sure all morphologies are represented
while i < (67631+count_per_type):
    get_fitscut(SGA['SGA_ID'][i],SGA['RA_LEDA'][i],SGA['DEC_LEDA'][i])
    img = fits.open('FITS/Gal{}.fits'.format(SGA['SGA_ID'][i]))
    images[j] = np.array(img[0].data)
    
    # Compile additional values into h5 file
    brickid[j] = SGA['BRICKID'][i]
    flux_g[j] = SGA['FLUX_G'][i]
    flux_r[j] = SGA['FLUX_R'][i]
    flux_z[j] = SGA['FLUX_Z'][i]
    fiberflux_g[j] = SGA['FIBERFLUX_G'][i]
    fiberflux_r[j] = SGA['FIBERFLUX_R'][i]
    fiberflux_z[j] = SGA['FIBERFLUX_R'][i]
    psfsize_g[j] = SGA['PSFSIZE_G'][i]
    psfsize_r[j] = SGA['PSFSIZE_R'][i]
    psfsize_z[j] = SGA['PSFSIZE_Z'][i] 
    psfdepth_g[j] = SGA['PSFDEPTH_G'][i] 
    psfdepth_r[j] = SGA['PSFDEPTH_R'][i]
    psfdepth_z[j] = SGA['PSFDEPTH_Z'][i] 
    nobs_g[j] = SGA['NOBS_G'][i]
    nobs_r[j] = SGA['NOBS_R'][i] 
    nobs_z[j] = SGA['NOBS_Z'][i] 
    fracin_g[j] = SGA['FRACIN_G'][i] 
    fracin_r[j] = SGA['FRACIN_R'][i] 
    fracin_z[j] = SGA['FRACIN_Z'][i]
    release[j] = SGA['RELEASE'][i]
    objid[j] = SGA['OBJID'][i]
    ebv[j] = SGA['EBV'][i]
    ra[j] = SGA['RA'][i]
    dec[j] = SGA['DEC'][i]
    mag_g[j] = SGA['G_MAG_SB26'][i]
    mag_r[j] = SGA['R_MAG_SB26'][i]
    mag_z[j] = SGA['Z_MAG_SB26'][i]
    
    time.sleep(0.1) #1 second pause per loop to not overwhelm server
    
    i+=1
    j+=1

In [121]:
#collect data for (MORPHTYPE = 3)
i=73621 #controls location within SGA catalog
j=(count_per_type*5) #controls location within the h5 file, separate so you cacn skip around in catalog to make sure all morphologies are represented
while i < (73621+count_per_type):
    get_fitscut(SGA['SGA_ID'][i],SGA['RA_LEDA'][i],SGA['DEC_LEDA'][i])
    img = fits.open('FITS/Gal{}.fits'.format(SGA['SGA_ID'][i]))
    images[j] = np.array(img[0].data)
    
    # Compile additional values into h5 file
    brickid[j] = SGA['BRICKID'][i]
    flux_g[j] = SGA['FLUX_G'][i]
    flux_r[j] = SGA['FLUX_R'][i]
    flux_z[j] = SGA['FLUX_Z'][i]
    fiberflux_g[j] = SGA['FIBERFLUX_G'][i]
    fiberflux_r[j] = SGA['FIBERFLUX_R'][i]
    fiberflux_z[j] = SGA['FIBERFLUX_R'][i]
    psfsize_g[j] = SGA['PSFSIZE_G'][i]
    psfsize_r[j] = SGA['PSFSIZE_R'][i]
    psfsize_z[j] = SGA['PSFSIZE_Z'][i] 
    psfdepth_g[j] = SGA['PSFDEPTH_G'][i] 
    psfdepth_r[j] = SGA['PSFDEPTH_R'][i]
    psfdepth_z[j] = SGA['PSFDEPTH_Z'][i] 
    nobs_g[j] = SGA['NOBS_G'][i]
    nobs_r[j] = SGA['NOBS_R'][i] 
    nobs_z[j] = SGA['NOBS_Z'][i] 
    fracin_g[j] = SGA['FRACIN_G'][i] 
    fracin_r[j] = SGA['FRACIN_R'][i] 
    fracin_z[j] = SGA['FRACIN_Z'][i]
    release[j] = SGA['RELEASE'][i]
    objid[j] = SGA['OBJID'][i]
    ebv[j] = SGA['EBV'][i]
    ra[j] = SGA['RA'][i]
    dec[j] = SGA['DEC'][i]
    mag_g[j] = SGA['G_MAG_SB26'][i]
    mag_r[j] = SGA['R_MAG_SB26'][i]
    mag_z[j] = SGA['Z_MAG_SB26'][i]
    
    i+=1
    j+=1

In [122]:
#collect data for (MORPHTYPE = 4)
i=84697 #controls location within SGA catalog
j=(count_per_type*6) #controls location within the h5 file, separate so you cacn skip around in catalog to make sure all morphologies are represented
while i < (84697+count_per_type):
    get_fitscut(SGA['SGA_ID'][i],SGA['RA_LEDA'][i],SGA['DEC_LEDA'][i])
    img = fits.open('FITS/Gal{}.fits'.format(SGA['SGA_ID'][i]))
    images[j] = np.array(img[0].data)
    
    # Compile additional values into h5 file
    brickid[j] = SGA['BRICKID'][i]
    flux_g[j] = SGA['FLUX_G'][i]
    flux_r[j] = SGA['FLUX_R'][i]
    flux_z[j] = SGA['FLUX_Z'][i]
    fiberflux_g[j] = SGA['FIBERFLUX_G'][i]
    fiberflux_r[j] = SGA['FIBERFLUX_R'][i]
    fiberflux_z[j] = SGA['FIBERFLUX_R'][i]
    psfsize_g[j] = SGA['PSFSIZE_G'][i]
    psfsize_r[j] = SGA['PSFSIZE_R'][i]
    psfsize_z[j] = SGA['PSFSIZE_Z'][i] 
    psfdepth_g[j] = SGA['PSFDEPTH_G'][i] 
    psfdepth_r[j] = SGA['PSFDEPTH_R'][i]
    psfdepth_z[j] = SGA['PSFDEPTH_Z'][i] 
    nobs_g[j] = SGA['NOBS_G'][i]
    nobs_r[j] = SGA['NOBS_R'][i] 
    nobs_z[j] = SGA['NOBS_Z'][i] 
    fracin_g[j] = SGA['FRACIN_G'][i] 
    fracin_r[j] = SGA['FRACIN_R'][i] 
    fracin_z[j] = SGA['FRACIN_Z'][i]
    release[j] = SGA['RELEASE'][i]
    objid[j] = SGA['OBJID'][i]
    ebv[j] = SGA['EBV'][i]
    ra[j] = SGA['RA'][i]
    dec[j] = SGA['DEC'][i]
    mag_g[j] = SGA['G_MAG_SB26'][i]
    mag_r[j] = SGA['R_MAG_SB26'][i]
    mag_z[j] = SGA['Z_MAG_SB26'][i]
    
    i+=1
    j+=1

In [123]:
#START HERE TO RUN
#collect data for (MORPHTYPE =5)
i=101991 #controls location within SGA catalog
j=(count_per_type*7) #controls location within the h5 file, separate so you cacn skip around in catalog to make sure all morphologies are represented
while i < (101991+count_per_type):
    get_fitscut(SGA['SGA_ID'][i],SGA['RA_LEDA'][i],SGA['DEC_LEDA'][i])
    img = fits.open('FITS/Gal{}.fits'.format(SGA['SGA_ID'][i]))
    images[j] = np.array(img[0].data)
    
    # Compile additional values into h5 file
    brickid[j] = SGA['BRICKID'][i]
    flux_g[j] = SGA['FLUX_G'][i]
    flux_r[j] = SGA['FLUX_R'][i]
    flux_z[j] = SGA['FLUX_Z'][i]
    fiberflux_g[j] = SGA['FIBERFLUX_G'][i]
    fiberflux_r[j] = SGA['FIBERFLUX_R'][i]
    fiberflux_z[j] = SGA['FIBERFLUX_R'][i]
    psfsize_g[j] = SGA['PSFSIZE_G'][i]
    psfsize_r[j] = SGA['PSFSIZE_R'][i]
    psfsize_z[j] = SGA['PSFSIZE_Z'][i] 
    psfdepth_g[j] = SGA['PSFDEPTH_G'][i] 
    psfdepth_r[j] = SGA['PSFDEPTH_R'][i]
    psfdepth_z[j] = SGA['PSFDEPTH_Z'][i] 
    nobs_g[j] = SGA['NOBS_G'][i]
    nobs_r[j] = SGA['NOBS_R'][i] 
    nobs_z[j] = SGA['NOBS_Z'][i] 
    fracin_g[j] = SGA['FRACIN_G'][i] 
    fracin_r[j] = SGA['FRACIN_R'][i] 
    fracin_z[j] = SGA['FRACIN_Z'][i]
    release[j] = SGA['RELEASE'][i]
    objid[j] = SGA['OBJID'][i]
    ebv[j] = SGA['EBV'][i]
    ra[j] = SGA['RA'][i]
    dec[j] = SGA['DEC'][i]
    mag_g[j] = SGA['G_MAG_SB26'][i]
    mag_r[j] = SGA['R_MAG_SB26'][i]
    mag_z[j] = SGA['Z_MAG_SB26'][i]
    
    i+=1
    j+=1

KeyboardInterrupt: 

In [ ]:
#collect data for (MORPHTYPE = 6)
i=130446 #controls location within SGA catalog
j=(count_per_type*8) #controls location within the h5 file, separate so you cacn skip around in catalog to make sure all morphologies are represented
while i < (130446+count_per_type):
    get_fitscut(SGA['SGA_ID'][i],SGA['RA_LEDA'][i],SGA['DEC_LEDA'][i])
    img = fits.open('FITS/Gal{}.fits'.format(SGA['SGA_ID'][i]))
    images[j] = np.array(img[0].data)
    
    # Compile additional values into h5 file
    brickid[j] = SGA['BRICKID'][i]
    flux_g[j] = SGA['FLUX_G'][i]
    flux_r[j] = SGA['FLUX_R'][i]
    flux_z[j] = SGA['FLUX_Z'][i]
    fiberflux_g[j] = SGA['FIBERFLUX_G'][i]
    fiberflux_r[j] = SGA['FIBERFLUX_R'][i]
    fiberflux_z[j] = SGA['FIBERFLUX_R'][i]
    psfsize_g[j] = SGA['PSFSIZE_G'][i]
    psfsize_r[j] = SGA['PSFSIZE_R'][i]
    psfsize_z[j] = SGA['PSFSIZE_Z'][i] 
    psfdepth_g[j] = SGA['PSFDEPTH_G'][i] 
    psfdepth_r[j] = SGA['PSFDEPTH_R'][i]
    psfdepth_z[j] = SGA['PSFDEPTH_Z'][i] 
    nobs_g[j] = SGA['NOBS_G'][i]
    nobs_r[j] = SGA['NOBS_R'][i] 
    nobs_z[j] = SGA['NOBS_Z'][i] 
    fracin_g[j] = SGA['FRACIN_G'][i] 
    fracin_r[j] = SGA['FRACIN_R'][i] 
    fracin_z[j] = SGA['FRACIN_Z'][i]
    release[j] = SGA['RELEASE'][i]
    objid[j] = SGA['OBJID'][i]
    ebv[j] = SGA['EBV'][i]
    ra[j] = SGA['RA'][i]
    dec[j] = SGA['DEC'][i]
    mag_g[j] = SGA['G_MAG_SB26'][i]
    mag_r[j] = SGA['R_MAG_SB26'][i]
    mag_z[j] = SGA['Z_MAG_SB26'][i]
    
    time.sleep(1) #1 second pause per loop to not overwhelm server
    
    i+=1
    j+=1

In [ ]:
#collect data for (MORPHTYPE = 7)
i=133828 #controls location within SGA catalog
j=(count_per_type*9) #controls location within the h5 file, separate so you cacn skip around in catalog to make sure all morphologies are represented
while i < (133828+count_per_type):
    get_fitscut(SGA['SGA_ID'][i],SGA['RA_LEDA'][i],SGA['DEC_LEDA'][i])
    img = fits.open('FITS/Gal{}.fits'.format(SGA['SGA_ID'][i]))
    images[j] = np.array(img[0].data)
    
    # Compile additional values into h5 file
    brickid[j] = SGA['BRICKID'][i]
    flux_g[j] = SGA['FLUX_G'][i]
    flux_r[j] = SGA['FLUX_R'][i]
    flux_z[j] = SGA['FLUX_Z'][i]
    fiberflux_g[j] = SGA['FIBERFLUX_G'][i]
    fiberflux_r[j] = SGA['FIBERFLUX_R'][i]
    fiberflux_z[j] = SGA['FIBERFLUX_R'][i]
    psfsize_g[j] = SGA['PSFSIZE_G'][i]
    psfsize_r[j] = SGA['PSFSIZE_R'][i]
    psfsize_z[j] = SGA['PSFSIZE_Z'][i] 
    psfdepth_g[j] = SGA['PSFDEPTH_G'][i] 
    psfdepth_r[j] = SGA['PSFDEPTH_R'][i]
    psfdepth_z[j] = SGA['PSFDEPTH_Z'][i] 
    nobs_g[j] = SGA['NOBS_G'][i]
    nobs_r[j] = SGA['NOBS_R'][i] 
    nobs_z[j] = SGA['NOBS_Z'][i] 
    fracin_g[j] = SGA['FRACIN_G'][i] 
    fracin_r[j] = SGA['FRACIN_R'][i] 
    fracin_z[j] = SGA['FRACIN_Z'][i]
    release[j] = SGA['RELEASE'][i]
    objid[j] = SGA['OBJID'][i]
    ebv[j] = SGA['EBV'][i]
    ra[j] = SGA['RA'][i]
    dec[j] = SGA['DEC'][i]
    mag_g[j] = SGA['G_MAG_SB26'][i]
    mag_r[j] = SGA['R_MAG_SB26'][i]
    mag_z[j] = SGA['Z_MAG_SB26'][i]
    
    time.sleep(1) #1 second pause per loop to not overwhelm server
    
    i+=1
    j+=1

In [ ]:
#collect data for (MORPHTYPE = 9)
i=136298 #controls location within SGA catalog
j=(count_per_type*10) #controls location within the h5 file, separate so you cacn skip around in catalog to make sure all morphologies are represented
while i < (136298+count_per_type):
    get_fitscut(SGA['SGA_ID'][i],SGA['RA_LEDA'][i],SGA['DEC_LEDA'][i])
    img = fits.open('FITS/Gal{}.fits'.format(SGA['SGA_ID'][i]))
    images[j] = np.array(img[0].data)
    
    # Compile additional values into h5 file
    brickid[j] = SGA['BRICKID'][i]
    flux_g[j] = SGA['FLUX_G'][i]
    flux_r[j] = SGA['FLUX_R'][i]
    flux_z[j] = SGA['FLUX_Z'][i]
    fiberflux_g[j] = SGA['FIBERFLUX_G'][i]
    fiberflux_r[j] = SGA['FIBERFLUX_R'][i]
    fiberflux_z[j] = SGA['FIBERFLUX_R'][i]
    psfsize_g[j] = SGA['PSFSIZE_G'][i]
    psfsize_r[j] = SGA['PSFSIZE_R'][i]
    psfsize_z[j] = SGA['PSFSIZE_Z'][i] 
    psfdepth_g[j] = SGA['PSFDEPTH_G'][i] 
    psfdepth_r[j] = SGA['PSFDEPTH_R'][i]
    psfdepth_z[j] = SGA['PSFDEPTH_Z'][i] 
    nobs_g[j] = SGA['NOBS_G'][i]
    nobs_r[j] = SGA['NOBS_R'][i] 
    nobs_z[j] = SGA['NOBS_Z'][i] 
    fracin_g[j] = SGA['FRACIN_G'][i] 
    fracin_r[j] = SGA['FRACIN_R'][i] 
    fracin_z[j] = SGA['FRACIN_Z'][i]
    release[j] = SGA['RELEASE'][i]
    objid[j] = SGA['OBJID'][i]
    ebv[j] = SGA['EBV'][i]
    ra[j] = SGA['RA'][i]
    dec[j] = SGA['DEC'][i]
    mag_g[j] = SGA['G_MAG_SB26'][i]
    mag_r[j] = SGA['R_MAG_SB26'][i]
    mag_z[j] = SGA['Z_MAG_SB26'][i]
    
    time.sleep(1) #1 second pause per loop to not overwhelm server
    
    i+=1
    j+=1

In [104]:
f = h5py.File('cutoutdata.hdf5').close